In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('arxiv.csv')
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,title,categories
0,704.0033,Convergence of the discrete dipole approximati...,"['physics', 'optics']"
1,704.0038,The discrete dipole approximation: an overview...,"['physics', 'optics']"
2,704.0479,The affine part of the Picard scheme,"['math', 'AG']"
3,704.1476,TeV-scale gravity in Horava-Witten theory on a...,['hep-th']
4,705.1155,State Vector Determination By A Single Trackin...,['astro-ph']


In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm

parser = en_core_web_sm.load()
parser.max_length = 7000000

In [4]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations]
    mytokens = " ".join([i for i in mytokens]) 
    return mytokens

In [5]:
import string

punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [8]:
from tqdm import tqdm

tqdm.pandas()
df['processed_text'] = df['title'].progress_apply(spacy_tokenizer)

100%|██████████| 100000/100000 [13:40<00:00, 121.81it/s]


In [9]:
def preprocess_categories(cat):
    cat = cat.replace('[', '')
    cat = cat.replace(']', '')
    cat = cat.replace("'", '')
    return cat

In [10]:
tqdm.pandas()
df['processed_categories'] = df['categories'].progress_apply(preprocess_categories)

100%|██████████| 100000/100000 [00:00<00:00, 466220.27it/s]


In [11]:
df['concat'] = df['processed_text'] + ' ' + df['processed_categories']
df.head()

,id,title,categories,processed_text,processed_categories,concat
0,704.0033,Convergence of the discrete dipole approximati...,"['physics', 'optics']",convergence discrete dipole approximation i. t...,"physics, optics",convergence discrete dipole approximation i. t...
1,704.0038,The discrete dipole approximation: an overview...,"['physics', 'optics']",discrete dipole approximation overview recent ...,"physics, optics",discrete dipole approximation overview recent ...
2,704.0479,The affine part of the Picard scheme,"['math', 'AG']",affine picard scheme,"math, AG","affine picard scheme math, AG"
3,704.1476,TeV-scale gravity in Horava-Witten theory on a...,['hep-th'],tev scale gravity horava witten theory compact...,hep-th,tev scale gravity horava witten theory compact...
4,705.1155,State Vector Determination By A Single Trackin...,['astro-ph'],state vector determination single tracking sat...,astro-ph,state vector determination single tracking sat...


In [22]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.concat)]

In [24]:
model = Doc2Vec(vector_size=64, min_count=1, epochs=20)
model.build_vocab(card_docs)

In [26]:
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)

In [29]:
card2vec = [model.infer_vector((df['concat'][i].split(' '))) for i in range(0,len(df['concat']))]

In [31]:
import numpy as np

dtv = np.array(card2vec).tolist()
df['vec'] = dtv
df.head()

,id,title,categories,processed_text,processed_categories,concat,vec
0,704.0033,Convergence of the discrete dipole approximati...,"['physics', 'optics']",convergence discrete dipole approximation i. t...,"physics, optics",convergence discrete dipole approximation i. t...,"[0.1269778460264206, 0.0676768496632576, -0.09..."
1,704.0038,The discrete dipole approximation: an overview...,"['physics', 'optics']",discrete dipole approximation overview recent ...,"physics, optics",discrete dipole approximation overview recent ...,"[0.14317379891872406, -0.04152209311723709, -0..."
2,704.0479,The affine part of the Picard scheme,"['math', 'AG']",affine picard scheme,"math, AG","affine picard scheme math, AG","[0.030805379152297974, -0.06239623948931694, -..."
3,704.1476,TeV-scale gravity in Horava-Witten theory on a...,['hep-th'],tev scale gravity horava witten theory compact...,hep-th,tev scale gravity horava witten theory compact...,"[-0.1107976958155632, 0.044458262622356415, -0..."
4,705.1155,State Vector Determination By A Single Trackin...,['astro-ph'],state vector determination single tracking sat...,astro-ph,state vector determination single tracking sat...,"[-0.1546483188867569, 0.19419126212596893, 0.0..."


In [32]:
df.to_csv('arxiv_vectors.zip', index=False) 